<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.4: Sequential Logic
**Prev: [Control Flow](2.3_control_flow.ipynb)**<br>
**Next: [FIR Filter](2.5_exercise.ipynb)**

## Motivation
没有状态，你就无法编写任何有意义的数字逻辑。没有状态，你就无法编写任何有意义的数字逻辑。没有状态，你就无法编写任何有意义的数字逻辑......

明白了吗？因为如果不存储中间结果，你就无法取得任何进展。

好吧，抛开这个冷笑话不谈，本模块将描述如何在 Chisel 中表达常见的时序模式。在本模块结束时，你应该能够在 Chisel 中实现和测试一个移位寄存器。

需要强调的是，这一部分可能不会让你印象深刻。Chisel 的强大之处不在于新的时序逻辑模式，而在于设计的参数化。在我们展示这种能力之前，我们必须了解这些时序模式是什么。因此，这一部分将向你展示 Chisel 能做的几乎所有 Verilog 能做的事情——你只需要学习 Chisel 的语法。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Registers
Chisel 中的基本有状态元素是寄存器，用 `Reg` 表示。
一个 `Reg` 保持其输出值直到时钟的上升沿，此时它接受其输入的值。
默认情况下，每个 Chisel `Module` 都有一个隐式时钟，设计中的每个寄存器都使用这个时钟。
这可以避免你在代码中始终指定相同的时钟。

<span style="color:blue">**Example: Using a Register**</span><br>
以下代码块实现了一个模块，该模块接受输入，增加 1，并将其连接为寄存器的输入。
*注意：隐式时钟可以被覆盖以用于多时钟设计。请参阅附录中的示例。*

In [ ]:
class RegisterModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = Reg(UInt(12.W))
  register := io.in + 1.U
  io.out := register
}

test(new RegisterModule) { c =>
  for (i <- 0 until 100) {
    c.io.in.poke(i.U)
    c.clock.step(1)
    c.io.out.expect((i + 1).U)
  }
}
println("SUCCESS!!")

寄存器通过调用 `Reg(tpe)` 创建，其中 `tpe` 是一个变量，用于编码我们想要的寄存器类型。
在这个例子中，`tpe` 是一个 12 位的 `UInt`。

看看上面的测试器在做什么。
在调用 `poke()` 和 `expect` 之间，有一个 `step(1)` 的调用。
这告诉测试框架时钟滴答一次，这将使寄存器将其输入传递到输出。

调用 `step(n)` 将使时钟滴答 `n` 次。

敏锐的观察者会注意到，以前测试组合逻辑的测试器没有调用 `step()`。这是因为在输入上调用 `poke()` 会立即通过组合逻辑传播更新的值。调用 `step()` 仅用于更新时序逻辑中的状态元素。

下面的代码块将显示 `RegisterModule` 生成的 Verilog。

注意：
* 模块有一个时钟（和复位）输入，这是你没有添加的——这是隐式时钟
* 变量 `register` 如预期那样显示为 `reg [11:0]`
* 有一个由 `ifdef Randomize` 分隔的块，在仿真开始前将寄存器初始化为某个随机变量
* `register` 在 `posedge clock` 上更新

In [ ]:
println(getVerilog(new RegisterModule))

一个重要的注意事项是 Chisel 区分类型（如 `UInt`）和硬件节点（如字面量 `2.U` 或 `myReg` 的输出）。虽然
```scala
val myReg = Reg(UInt(2.W))
```
是合法的，因为寄存器需要一个数据类型作为模型，
```scala
val myReg = Reg(2.U)
```
则是错误的，因为 `2.U` 已经是一个硬件节点，不能用作模型。

<span style="color:blue">**Example: RegNext**</span><br>
Chisel 为具有简单输入连接的寄存器提供了一个方便的寄存器对象。之前的 `Module` 可以简化为以下 `Module`。注意这次我们不需要指定寄存器的位宽。它从寄存器的输出连接中推断出来，在这个例子中是 `io.out`。

In [ ]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}

test(new RegNextModule) { c =>
  for (i <- 0 until 100) {
    c.io.in.poke(i.U)
    c.clock.step(1)
    c.io.out.expect((i + 1).U)
  }
}
println("SUCCESS!!")

The Verilog looks almost the same as before, though the register name is generated instead of explicity defined.

In [ ]:
println(getVerilog(new RegNextModule))

---
# `RegInit`

`RegisterModule` 中的寄存器被初始化为模拟的随机数据。
除非另有说明，否则寄存器没有复位值（或复位）。
创建一个复位到给定值的寄存器的方法是使用 `RegInit`。

例如，一个初始化为零的12位寄存器可以用以下方式创建。
以下两个版本都是有效的并且实现相同的功能：
```scala
val myReg = RegInit(UInt(12.W), 0.U)
val myReg = RegInit(0.U(12.W))
```

第一个版本有两个参数。
第一个参数是一个类型节点，指定数据类型及其宽度。
第二个参数是一个硬件节点，指定复位值，在本例中为0。

第二个版本有一个参数。
它是一个硬件节点，指定复位值，通常是 `0.U`。

<span style="color:blue">**Example: Initialized Register** </span><br>
下面演示了使用 `RegInit()`，初始化为零。

In [ ]:
class RegInitModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = RegInit(0.U(12.W))
  register := io.in + 1.U
  io.out := register
}

println(getVerilog(new RegInitModule))

请注意，生成的 Verilog 现在有一个检查 `if (reset)` 的块，用于将寄存器复位为 0。
还要注意，这在 `always @(posedge clock)` 块内。
Chisel 的隐式复位是高电平有效且同步的。
在调用复位之前，寄存器仍会初始化为随机数据。
`PeekPokeTesters` 总是在运行测试之前调用复位，但你也可以使用 `reset(n)` 函数手动调用复位，其中复位在 `n` 个周期内为高电平。

---
# Control Flow
在控制流方面，寄存器与导线非常相似。
它们具有最后连接语义，并且可以使用 `when`、`elsewhen` 和 `otherwise` 进行条件赋值。

<span style="color:blue">**Example: Register Control Flow**</span><br>
以下示例使用条件寄存器赋值找到一系列输入中的最大值。

In [ ]:
class FindMax extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(10.W))
    val max = Output(UInt(10.W))
  })

  val max = RegInit(0.U(10.W))
  when (io.in > max) {
    max := io.in
  }
  io.max := max
}

test(new FindMax) { c =>
    c.io.max.expect(0.U)
    c.io.in.poke(1.U)
    c.clock.step(1)
    c.io.max.expect(1.U)
    c.io.in.poke(3.U)
    c.clock.step(1)
    c.io.max.expect(3.U)
    c.io.in.poke(2.U)
    c.clock.step(1)
    c.io.max.expect(3.U)
    c.io.in.poke(24.U)
    c.clock.step(1)
    c.io.max.expect(24.U)
}
println("SUCCESS!!")

---
# Other Register Examples

对寄存器调用的操作是在寄存器的**输出**上执行的，操作的种类取决于寄存器的类型。
这意味着你可以写
```scala
val reg: UInt = Reg(UInt(4.W))
```
这意味着值 `reg` 的类型是 `UInt`，你可以对 `UInt` 进行正常操作，比如 `+`、`-` 等。

你并不限于在寄存器中使用 `UInt`，你可以使用任何 `chisel3.Data` 基类型的子类。这包括用于有符号整数的 `SInt` 以及很多其他东西。

<span style="color:blue">**Example: Comb Filter**</span><br>
以下示例显示了一个梳状滤波器。

In [ ]:
class Comb extends Module {
  val io = IO(new Bundle {
    val in  = Input(SInt(12.W))
    val out = Output(SInt(12.W))
  })

  val delay: SInt = Reg(SInt(12.W))
  delay := io.in
  io.out := io.in - delay
}
println(getVerilog(new Comb))

---
# Exercises
<span style="color:red">**Exercise: Shift Register**</span><br>
基于你新学到的寄存器知识，构建一个实现线性反馈移位寄存器（LFSR）的模块。具体要求：
- 每个元素宽度为一位。
- 有一个4位输出信号。
- 接受一个输入位，这是移位寄存器的下一个值。
- 输出移位寄存器的并行输出，最高有效位是移位寄存器的最后一个元素，最低有效位是移位寄存器的第一个元素。`Cat` 可能会派上用场。
- **输出初始化为 `b0001`。**
- 每个时钟周期移位一次（无使能信号）。
- **注意在 Chisel 中，子位分配是不合法的**；类似 `out(0) := in` 的操作是不可行的。

<img src="images/shifter4.svg" alt="shift register figure" style="width: 450px" />

以下提供了一个基本的模块框架、测试向量和 Driver 调用。第一个寄存器已经为你提供。

In [ ]:
class MyShiftRegister(val init: Int = 1) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), init.U)

  ???
}

test(new MyShiftRegister()) { c =>
  var state = c.init
  for (i <- 0 until 10) {
    // poke in LSB of i (i % 2)
    c.io.in.poke(((i % 2) != 0).B)
    // update expected state
    state = ((state * 2) + (i % 2)) & 0xf
    c.clock.step(1)
    c.io.out.expect(state.U)
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  state := nextState
  io.out := state
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Shift Register (Optional)**</span><br>
Write a shift register that is parameterized by its delay (`n`), its initial value (`init`), and also has an enable input signal (`en`).

In [ ]:
// n is the output width (number of delays - 1)
// init state to init
class MyOptionalShiftRegister(val n: Int, val init: BigInt = 1) extends Module {
  val io = IO(new Bundle {
    val en  = Input(Bool())
    val in  = Input(Bool())
    val out = Output(UInt(n.W))
  })

  val state = RegInit(init.U(n.W))

  ???
}

// test different depths
for (i <- Seq(3, 4, 8, 24, 65)) {
  println(s"Testing n=$i")
  test(new MyOptionalShiftRegister(n = i)) { c =>
    val inSeq = Seq(0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
    var state = c.init
    var i = 0
    c.io.en.poke(true.B)
    while (i < 10 * c.n) {
      // poke in repeated inSeq
      val toPoke = inSeq(i % inSeq.length)
      c.io.in.poke((toPoke != 0).B)
      // update expected state
      state = ((state * 2) + toPoke) & BigInt("1"*c.n, 2)
      c.clock.step(1)
      c.io.out.expect(state.U)

      i += 1
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  when (io.en) {
    state  := nextState
  }
  io.out := state
</pre></article></div></section></div>

---
# Appendix: Explicit clock and reset
Chisel 模块有一个默认的时钟和复位，这些时钟和复位被每个在它们内部创建的寄存器隐式使用。
有时你可能希望能够覆盖这种默认行为；也许你有一个生成时钟或复位信号的黑盒，或者你有一个多时钟设计。

Chisel 提供了用于处理这些情况的构造。
可以分别或一起使用 `withClock() {}`、`withReset() {}` 和 `withClockAndReset() {}` 来覆盖时钟和复位。
以下代码块将展示使用这些函数的示例。

需要注意的一点是，`reset`（截至本教程编写时）始终是同步的，并且类型为 `Bool`。
时钟在 Chisel 中有自己的类型（`Clock`），应如此声明。
*`Bool` 可以通过调用 `asClock()` 转换为 `Clock`，但你应该小心不要做一些愚蠢的事情。*

还要注意，`chisel-testers` 目前不完全支持多时钟设计。

<span style="color:blue">**Example: Multi-Clock Module**</span><br>
一个具有多个时钟和复位信号的模块。

In [ ]:
// we need to import multi-clock features
import chisel3.experimental.{withClock, withReset, withClockAndReset}

class ClockExamples extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(10.W))
    val alternateReset    = Input(Bool())
    val alternateClock    = Input(Clock())
    val outImplicit       = Output(UInt())
    val outAlternateReset = Output(UInt())
    val outAlternateClock = Output(UInt())
    val outAlternateBoth  = Output(UInt())
  })

  val imp = RegInit(0.U(10.W))
  imp := io.in
  io.outImplicit := imp

  withReset(io.alternateReset) {
    // everything in this scope with have alternateReset as the reset
    val altRst = RegInit(0.U(10.W))
    altRst := io.in
    io.outAlternateReset := altRst
  }

  withClock(io.alternateClock) {
    val altClk = RegInit(0.U(10.W))
    altClk := io.in
    io.outAlternateClock := altClk
  }

  withClockAndReset(io.alternateClock, io.alternateReset) {
    val alt = RegInit(0.U(10.W))
    alt := io.in
    io.outAlternateBoth := alt
  }
}

println(getVerilog(new ClockExamples))

---
# Wrap Up
完成这一部分的学习，做得很好！！你现在已经学会了如何在 Chisel 中创建寄存器和编写时序逻辑，这意味着你有足够的基本构建模块来编写真实电路了。

下一部分将把我们学到的所有内容结合到一个示例中！如果你需要更多的鼓励，请记住这位 Chisel 专家的话：

![BobRoss](http://i.qkme.me/3qbd5u.jpg)

---
# You're done!

[Return to the top.](#top)